In [2]:
import json
from collections import Counter
import pandas as pd
import numpy as np
import glob
from konlpy.tag import Kkma
import re
import tqdm

import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
import warnings
warnings.filterwarnings('ignore')

import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt

In [4]:
glob.glob('./data_ko/*.json')

['./data_ko/ko_review_jeju.json',
 './data_ko/ko_review_jeonju.json',
 './data_ko/ko_review_daejeon.json',
 './data_ko/ko_review_daegu.json',
 './data_ko/ko_review_seogwipo.json',
 './data_ko/ko_review_yeosu.json',
 './data_ko/ko_review_gangneung.json',
 './data_ko/ko_review_seoul.json',
 './data_ko/ko_review_gwangju.json']

In [28]:
review_lst = ['./data_ko/ko_review_jeju.json',
 './data_ko/ko_review_seogwipo.json',
 './data_ko/ko_review_gangneung.json',
 './data_ko/ko_review_yeosu.json']

In [29]:
review = {}
for file in review_lst:
    with open(file,'r',encoding='utf-8') as fp:
        review.update(json.load(fp))

In [30]:
review_total = []
for home_id,rev_lst in review.items():
    for rev in rev_lst :
        if len(rev) > 1 :
            review_total.append(rev)
            
print('number of reviews : ', len(review_total))

number of reviews :  122166


# Text preprocessing

In [ ]:
kkma = Kkma()
for rev in review_total :
    rev[1] = kkma.sentences(rev[1])

In [ ]:
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자

for rev in review_total : 
    sen_lst = []
    for r in rev[1] :
        sen = hangul.sub('', r)
        sen = sen.strip().replace("ㅎ", "").replace("ㅠ", "").replace("ㅜ", "")
        sen_lst.append(sen)
        rev[1] = sen_lst 

In [ ]:
for rev in review_total : 
    sen_lst = []
    for r in rev[1] :
        sen = r.strip()
        if len(sen) > 1 :
            sen_lst.append(sen)
        rev[1] = sen_lst 

In [ ]:
with open('review_split_sea.json','w',encoding='utf-8') as fp:
        json.dump(review_total,fp)

# Pos tagging

In [4]:
with open('review_split_sea.json') as data_file:    
    review_total = json.load(data_file)

In [7]:
for rev in tqdm.tqdm(review_total) : 
    sen_lst = []
    for r in rev[1] :
        sen = kkma.pos(r)
        sen_lst.append(sen)
    rev[1] = sen_lst 

100%|██████████| 122166/122166 [46:32<00:00, 43.75it/s] 


In [8]:
for rev in review_total : 
    sen_lst = []
    for sen in rev[1] :
        w_lst = []
        for (word,pos) in sen :
            if pos in ['NN','NNG','NNP','NP','VA','VV','XR'] :
                w_lst.append(word)
        sen_lst.append(w_lst)
        rev[1] = sen_lst

In [9]:
review_total[3000]

[5,
 [['알려주', '아깝', '정도', '혼자', '알', '숙소'],
  ['방', '구조', '풍', '디테일', '소품', '아름답'],
  ['바다',
   '바라보',
   '내리',
   '마시',
   '있',
   '커피',
   '도구',
   '넷',
   '보',
   '있',
   '오',
   '책',
   '있',
   '하루',
   '종일',
   '숙소',
   '있',
   '정도',
   '들어오',
   '나가',
   '이날',
   '일정',
   '게',
   '좋',
   '같'],
  ['멋지', '숙소', '머물', '따뜻', '사장님', '좋'],
  ['제', '들르']]]

In [10]:
with open('review_NV_sea.json','w',encoding='utf-8') as fp:
        json.dump(review_total,fp)

In [15]:
token = []
for r in review_total :
    for s in r[1] :
        for w in s :
            token.append(w)
        
token_cnt = Counter(token)
sorted(token_cnt.items(),key=lambda kv:kv[1],reverse=True)[:30]

[('좋', 131252),
 ('있', 85081),
 ('숙소', 72305),
 ('하', 58458),
 ('오', 30645),
 ('깨끗하', 30024),
 ('친절', 29751),
 ('여행', 27876),
 ('호스트', 25655),
 ('곳', 25349),
 ('같', 24484),
 ('되', 23997),
 ('집', 23990),
 ('위치', 23705),
 ('가', 23493),
 ('분', 22311),
 ('없', 22239),
 ('깔끔', 20146),
 ('갈', 19366),
 ('다음', 19013),
 ('쉬', 18984),
 ('방', 18776),
 ('감사', 18038),
 ('조용', 17069),
 ('편하', 16986),
 ('보', 16467),
 ('가족', 15073),
 ('가깝', 14663),
 ('이용', 13948),
 ('만족', 13346)]

In [18]:
stopWords = ['있','잇','하','되','분','없']

for rev in review_total : 
    sen_lst = []
    for sen in rev[1] :
        no_stopword = [w for w in sen if not w in stopWords]
        sen_lst.append(no_stopword)
        rev[1] = sen_lst

# LDA

In [21]:
texts = []
for rev in review_total:
    for sen in rev[1] :
         if len(sen)>0 :
            texts.append(sen)
        
print('문장 갯수 :',len(texts))
texts[:3]

문장 갯수 : 441178


[['감사'], ['가격', '대비', '훌륭'], ['다르', '후기', '같', '곳']]

In [22]:
dic = corpora.Dictionary(texts)

In [23]:
corpus = [dic.doc2bow(text) for text in texts]

In [24]:
import time
start = time.time()

n = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = n, id2word = dic) 
print('걸린시간(m) :',(time.time()-start)/60)

걸린시간(m) : 0.9044737776120504


In [25]:
ldamodel.show_topics(num_words=10) #토픽을 구성하는 주요단어들

[(0,
  '0.035*"가" + 0.031*"오" + 0.031*"좋" + 0.029*"쉬" + 0.028*"다음" + 0.026*"숙소" + 0.023*"가깝" + 0.023*"곳" + 0.021*"갈" + 0.018*"위치"'),
 (1,
  '0.027*"추천" + 0.026*"감사" + 0.022*"먹" + 0.018*"조식" + 0.017*"아침" + 0.017*"쓰" + 0.016*"맛있" + 0.016*"준비" + 0.015*"듣" + 0.014*"드리"'),
 (2,
  '0.036*"좋" + 0.020*"보" + 0.019*"방" + 0.019*"바다" + 0.019*"보이" + 0.013*"화장실" + 0.012*"크" + 0.012*"불편" + 0.012*"나오" + 0.012*"점"'),
 (3,
  '0.111*"좋" + 0.078*"숙소" + 0.047*"깨끗하" + 0.032*"깔끔" + 0.028*"조용" + 0.020*"예쁘" + 0.020*"사진" + 0.019*"위치" + 0.019*"집" + 0.018*"넓"'),
 (4,
  '0.050*"친절" + 0.045*"호스트" + 0.031*"좋" + 0.029*"여행" + 0.021*"지내" + 0.017*"시간" + 0.017*"가족" + 0.015*"오" + 0.013*"배려" + 0.012*"사장님"')]

In [27]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dic)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      22.095922        1       1  0.206706 -0.107320
3      20.793522        1       2  0.173555  0.195597
2      20.233349        1       3 -0.056484  0.221461
4      20.043314        1       4  0.035677 -0.279883
1      16.833897        1       5 -0.359454 -0.029854, topic_info=      Category           Freq Term          Total  loglift  logprob
14691  Default   70001.000000   숙소   70001.000000  30.0000  30.0000
11594  Default   29253.000000   친절   29253.000000  29.0000  29.0000
16023  Default  128322.000000    좋  128322.000000  28.0000  28.0000
2612   Default   28974.000000  깨끗하   28974.000000  27.0000  27.0000
7164   Default   26227.000000  호스트   26227.000000  26.0000  26.0000
16304  Default   22473.000000    가   22473.000000  25.0000  25.0000
18056  Default   19792.000000   깔끔   19792.000000  24.0000  24.0000
16674  Default   18757.000000    쉬   18757.000000  23.0000  23.0000
4503   Default   18108.000000   다음   18108.000000  22.0000  22.0000
16854  Default   17076.000000   조용   17076.000000  21.0000  21.0000
10491  Default   29157.000000    오   29157.000000  20.0000  20.0000
14666  Default   28295.000000   여행   28295.000000  19.0000  19.0000
5907   Default   13318.000000   추천   13318.000000  18.0000  18.0000
166    Default   15007.000000   가깝   15007.000000  17.0000  17.0000
11582  Default   18515.000000   감사   18515.000000  16.0000  16.0000
1087   Default   24500.000000    곳   24500.000000  15.0000  15.0000
18866  Default   12356.000000   지내   12356.000000  14.0000  14.0000
11513  Default   23509.000000   위치   23509.000000  13.0000  13.0000
9300   Default   10923.000000    먹   10923.000000  12.0000  12.0000
15471  Default   12102.000000   예쁘   12102.000000  11.0000  11.0000
262    Default   12057.000000   사진   12057.000000  10.0000  10.0000
8060   Default   11567.000000   바다   11567.000000   9.0000   9.0000
1500   Default   11026.000000   보이   11026.000000   8.0000   8.0000
4      Default   18862.000000    갈   18862.000000   7.0000   7.0000
6842   Default   11229.000000    넓   11229.000000   6.0000   6.0000
10343  Default   19037.000000    방   19037.000000   5.0000   5.0000
7431   Default    8896.000000   조식    8896.000000   4.0000   4.0000
4220   Default    9834.000000   시간    9834.000000   3.0000   3.0000
13028  Default    9950.000000   청결    9950.000000   2.0000   2.0000
18476  Default   10230.000000  제주도   10230.000000   1.0000   1.0000
...        ...            ...  ...            ...      ...      ...
9972    Topic5    3123.144043  하우스    3124.232422   1.7814  -5.0655
5866    Topic5    2235.548096   식기    2236.351074   1.7814  -5.3998
687     Topic5    2213.076172  고양이    2213.889160   1.7814  -5.4099
8287    Topic5    2167.785645   귀엽    2168.590332   1.7814  -5.4306
4531    Topic5    2029.172852   용품    2029.975952   1.7814  -5.4967
6215    Topic5    2009.099731  강아지    2009.907593   1.7814  -5.5066
1369    Topic5    2008.139038   이상    2008.950684   1.7814  -5.5071
9873    Topic5    1988.685059   청소    1989.498291   1.7814  -5.5168
10067   Topic5    1965.768066  세탁기    1966.574829   1.7814  -5.5284
10510   Topic5    1938.958496   음식    1939.762695   1.7814  -5.5421
17054   Topic5    1893.437866    드    1894.244263   1.7813  -5.5659
6198    Topic5    1840.854980   해먹    1841.660767   1.7813  -5.5941
6643    Topic5    1792.364502   정성    1793.171631   1.7813  -5.6208
16539   Topic5    1700.449341   도구    1701.251465   1.7813  -5.6734
1796    Topic5    1709.368774   계획    1710.187500   1.7813  -5.6682
19101   Topic5    6742.848633   드리    6750.808594   1.7806  -4.2958
13946   Topic5    4413.082520   커피    4436.431641   1.7765  -4.7197
4544    Topic5    7575.674805    듣    7658.630859   1.7709  -4.1794
2245    Topic5    5679.566895   필요    5725.318359   1.7738  -4.4674
5059    Topic5    2187.743896    어    2195.732666   1.7781  -5.4214
5048    Topic5    8618.604492   아침   11587.

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel.load('lda_sea.lda')

In [ ]:
ldamodel.save('lda_sea.lda')

In [ ]:
pyLDAvis.save_html(vis,'LDA_sea.html')